## Nota sobre los cambios respecto al notebook original

En esta versión del proyecto, he realizado varios cambios respecto al código base proporcionado:

- **Uso de Polars en vez de pandas:**  
  He optado por seguir utilizando Polars para la manipulación de datos en Python, ya que es una librería más eficiente y rápida para grandes volúmenes de datos, además de ofrecer una sintaxis moderna y muy potente para análisis y transformaciones.

- **Conexión a la base de datos con SQLAlchemy:**  
  En lugar de conectores más simples como `mysql-connector` o `pymysql`, sigo utilizando SQLAlchemy, que es más flexible y profesional, y permite una mejor gestión de conexiones, transacciones y portabilidad del código.

- **Gestión de credenciales con dotenv:**  
  Para evitar exponer contraseñas y parámetros sensibles en el código, he usado la librería `python-dotenv` y guardo los datos de conexión en un archivo `.env`, que está excluido del repositorio con `.gitignore` para mayor seguridad.

- **Manejo de errores y código más robusto:**  
  Se ha añadido control de errores y mensajes informativos en las funciones de consulta para asegurar que cualquier fallo en la conexión o en las queries sea fácil de identificar y depurar, acción que ayuda mucho para identificar errores de todo tipo.

En definitiva, estos cambios son para seguir un poco la lógica del primer notebook, y seguir usando las mismas librerías y dependencias.


In [98]:
# Importamos todas las dependencias necesarias (a priori)

import pandas as pd
import numpy as np
import polars as pl

from dotenv import load_dotenv
import os

import datetime

from meteostat import Point, Daily
import sqlalchemy as sa
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

import plotly.express as px
import matplotlib.pyplot as plt
import missingno as msno

## 1. Consulta a la base de datos

In [68]:
# Paso hecho anteriormente en el notebook 02, he visto que aquí también está así que vuelvo a hacer este paso
class DatabaseConnection:

    # Información necesaria para establecer la conexión a la BBDD
    def __init__(self):
        load_dotenv()  # Variables de entorno
        port = 3306  # Puerto por defecto de MySQL
        user = os.getenv("DB_USER")
        password = os.getenv("DB_PASSWORD")
        host = os.getenv("DB_HOST")
        nombre_base_datos = os.getenv("DB_NAME") 
        self.engine = create_engine(
            f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{nombre_base_datos}"
        )

    # Función cuyo cometido es consultar una query a la BBDD y devolverla como un dataframe de Polars
    def query_to_polars(self, sql_query):
        try:
            with self.engine.connect() as conn:
                result = conn.execute(text(sql_query))
                columns = result.keys()
                rows = result.fetchall()
                if not rows:
                    print("Consulta ejecutada correctamente pero no hay resultados.")
                    return pl.DataFrame(schema=columns)
                print("Consulta ejecutada correctamente.")
                return pl.DataFrame(rows, schema=columns)
        except SQLAlchemyError as e:
            print(f"Error de SQLAlchemy al ejecutar la consulta:\{e}")
            return None
        except Exception as e:
            print(f"Error inesperado:\{e}")
            return None

# Usamos y probamos la clase
db = DatabaseConnection()

query = """
SELECT *
FROM ventas_diarias_estudio
WHERE ARTICULO = '3960'
"""

ventas_prueba = db.query_to_polars(query)
ventas_prueba




Consulta ejecutada correctamente.


familia,tipo,fechaVenta,festivo,articulo,precio,orden_articulo_familia,in_fecha_estudio,cantidad,importe
str,str,date,str,str,f64,i64,str,f64,f64
"""BOLLERIA""","""VENTA""",2021-05-01,null,"""3960""",2.318,1,"""S""",2412.0,5591.015968
"""BOLLERIA""","""VENTA""",2021-05-02,"""Día de la Madre""","""3960""",2.318,1,"""S""",2214.0,5132.051977
"""BOLLERIA""","""VENTA""",2021-05-03,"""Día de la Cruz""","""3960""",2.318,1,"""S""",1368.0,3171.023967
"""BOLLERIA""","""VENTA""",2021-05-04,null,"""3960""",2.318,1,"""S""",1422.0,3296.195992
"""BOLLERIA""","""VENTA""",2021-05-05,null,"""3960""",2.318,1,"""S""",1728.0,4005.504001
…,…,…,…,…,…,…,…,…,…
"""BOLLERIA""","""VENTA""",2023-04-26,null,"""3960""",3.273,1,"""S""",972.0,3181.356079
"""BOLLERIA""","""VENTA""",2023-04-27,null,"""3960""",3.273,1,"""S""",1296.0,4241.808105
"""BOLLERIA""","""VENTA""",2023-04-28,null,"""3960""",3.273,1,"""S""",1098.0,3593.754089


In [ ]:
# La Query está adaptada, usa mi base y mi tabla correcta (nombre)
# IMP: se omiten las columnas irrelevantes -> 'in_fecha_estudio' y 'tipo'
query_daily_and_top = """
SELECT  familia,
        articulo,
        fechaVenta AS fecha_venta,
        festivo,
        precio,
        cantidad,
        importe,
        orden_articulo_familia

FROM ventas_diarias_estudio_completo
"""
# !!ESCRIBIR EL "COMPLETO"!! tenia errores en la carga de datos por no añadir el "completo" en la consulta SQL

# Instanciar la clase con la contraseña de la BBDD (mejor ocultar la contraseña)
db = DatabaseConnection()

# Consultar la query anterior y devolverla como un dataframe de Polars
ventas = db.query_to_polars(query_daily_and_top)

# Convertir 'articulo' a entero en Polars, ya cambié el formato a "Date" al insertar las tablas en el primer notebook
ventas = ventas.with_columns([
    pl.col("articulo").cast(pl.Int64)
])

Consulta ejecutada correctamente.


In [76]:
ventas

familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia
str,i64,date,str,f64,f64,f64,i64
"""BOLLERIA""",3880,2021-05-01,null,2.591,1710.0,4430.609985,3
"""BOLLERIA""",3960,2021-05-01,null,2.318,2412.0,5591.015968,1
"""BOLLERIA""",5803,2021-05-01,null,2.727,1422.0,3877.793884,5
"""BOLLERIA""",6286,2021-05-01,null,3.136,990.0,3104.640038,4
"""BOLLERIA""",6425,2021-05-01,null,31.364,190.349997,5970.1325,2
…,…,…,…,…,…,…,…
"""PASTELERIA""",6523,2023-05-17,null,40.772999,72.0,2935.656006,3
"""PASTELERIA""",5403,2023-05-18,null,40.772999,46.800001,1908.174042,2
"""PASTELERIA""",5404,2023-05-18,null,43.5,108.0,4698.0,1


In [77]:
# Voy a realizar una comprobación de los tipos de las columnas (experimentación con Polars)

for col, dtype in zip(ventas.columns, ventas.dtypes):
    print(f"{col}: {dtype}")

familia: String
articulo: Int64
fecha_venta: Date
festivo: String
precio: Float64
cantidad: Float64
importe: Float64
orden_articulo_familia: Int64


In [78]:
# Compruebo la columna festivo, no recordaba el formato y ahora veo el porqué del "String"
print(ventas["festivo"].head(10))

shape: (10,)
Series: 'festivo' [str]
[
	null
	null
	null
	null
	null
	"Día de la Madre"
	"Día de la Madre"
	"Día de la Madre"
	"Día de la Madre"
	"Día de la Madre"
]


In [79]:
# El método ".select" sería lo correcto con Polars
print(ventas.select("festivo").unique())

shape: (27, 1)
┌─────────────────────────────────┐
│ festivo                         │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ Lunes Santo                     │
│ Día de la Victoria              │
│ Día de Todos los Santos         │
│ Feria de Málaga                 │
│ Domingo de Resurrección         │
│ …                               │
│ Jueves Santo                    │
│ Día de la Hispanidad (Fiesta N… │
│ Domingo de Ramos                │
│ Año Nuevo                       │
│ Día del Padre                   │
└─────────────────────────────────┘


## 2. Consulta a [meteostat API](https://dev.meteostat.net/python/daily.html)

Variables que se obtienen:

- **tavg** -> The average air temperature in °C
- **tmin** -> The minimum air temperature in °C
- **tmax** -> The maximum air temperature in °C
- **prcp** -> The daily precipitation total in mm
- **wdir** -> The average wind direction in degrees (°)
- **wspd** -> The average wind speed in km/h
- **pres** -> The average sea-level air pressure in hPa

In [80]:
class DailyWeatherData:

    # Localización de la tienda de la Panadería Salvador Echeverría
    ECHEVERRIA_SHOP = Point(36.721477644071705, -4.363132134392174)

    # Columnas identificadas como importantes
    IMP_COLUMNS = ['tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres']

    # Definición del horizonte temporal de la consulta a la API
    def __init__(self, start, end=None):
        self.start = DailyWeatherData._to_datetime(start)
        if end is not None:
            self.end = DailyWeatherData._to_datetime(end)
        else:
            self.end = datetime.datetime.now()

    # He tenido errores de formato datetime.datetime y datetime.date (end y start), con la siguiente función soluciono el problema (creo que es tema de usar Polars)
    @staticmethod  # Uso un @staticmethod para llamar la clase directamente sin necesidad de crear un objeto antes
    def _to_datetime(dt):
        if isinstance(dt, datetime.datetime):
            return dt
        if isinstance(dt, datetime.date):  # Si es datetime.date, convierto SIEMPRE a datetime.datetime
            return datetime.datetime(dt.year, dt.month, dt.day)
        if hasattr(dt, "to_pydatetime"):  # Por si es Polars
            res = dt.to_pydatetime()
            # Por si meteostat devuelve date igualmente
            if isinstance(res, datetime.date) and not isinstance(res, datetime.datetime):
                return datetime.datetime(res.year, res.month, res.day)
            return res
        raise TypeError(f"No puedo convertir {dt} a datetime.datetime")

    # Se establece que la consulta será diaria, se seleccionan todas las columnas imps y se añade un suff.
    def get_weather_data(self, as_polars=True):   # Añado un argumento extra, recibiré un df de Pandas y tengo que convertirlo a Polars
        start_dt = self._to_datetime(self.start)
        end_dt = self._to_datetime(self.end)
        print("start_dt:", start_dt, type(start_dt))
        print("end_dt:", end_dt, type(end_dt))
        daily_data = Daily(self.ECHEVERRIA_SHOP, start_dt, end_dt)
        data = daily_data.fetch()  # Aquí devuelve un df de Pandas
        df = data[self.IMP_COLUMNS]
        df.columns = [f"{col}_w" for col in df.columns]  # Añado el sufijo para evitar conflictos con algun merge (w=weather)
        df = df.reset_index()
        # Aquí cambio a Polars
        if as_polars:
            return pl.from_pandas(df)
        else:
            return df
        
        


### Nota sobre fechas y compatibilidad Polars/Meteostat

Al trabajar con **Polars** para el análisis y luego tirar de la API de **Meteostat** para meter datos de tiempo, he tenido que pelearme un poco con los tipos de fecha.

- Cuando sacas el mínimo o máximo de una columna de fechas con Polars, lo que te devuelve es un **`datetime.date`**.
- Pero la librería Meteostat, para sus consultas, quiere **sí o sí un `datetime.datetime`** (no es lo mismo), porque por dentro resta con `datetime.now()` (que es `datetime.datetime`).

¿El resultado si no lo conviertes? Te comes un error del tipo:

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'datetime.date'

Con pandas este problema sale menos, porqué suele devolver un tipo de fecha más compatible, pero aun así puede pasar.  
Por eso, lo más seguro es **convertir todo a `datetime.datetime` antes de llamar a Meteostat**, y así te ahorras sustos y funciona todo bien, uses la librería que uses para tus datos.

> **Resumen**:  
> Si mezclas varias librerías, revisa siempre qué tipo de fecha manejas antes de combinarlas… así te evitas quebraderos de cabeza como este 😉



In [87]:
start_ventas = ventas["fecha_venta"].min()
end_ventas = ventas["fecha_venta"].max()

print(start_ventas)
print(end_ventas)

2021-05-01
2023-05-18


In [88]:
# Vamos a usar las fechas de inicio y fin dónde haya ventas
start_date = datetime.date(2021, 5, 1)
end_date = datetime.date(2023, 5, 18)

weather_getter = DailyWeatherData(start=start_date, end=end_date)
weather = weather_getter.get_weather_data(as_polars=True)
weather



start_dt: 2021-05-01 00:00:00 <class 'datetime.datetime'>
end_dt: 2023-05-18 00:00:00 <class 'datetime.datetime'>


time,tavg_w,tmin_w,tmax_w,prcp_w,wdir_w,wspd_w,pres_w
datetime[ns],f64,f64,f64,f64,f64,f64,f64
2021-05-01 00:00:00,17.2,11.9,22.8,0.0,null,14.2,1014.0
2021-05-02 00:00:00,16.3,10.8,23.2,0.0,null,15.7,1015.9
2021-05-03 00:00:00,15.3,12.7,20.1,6.5,null,9.7,1017.7
2021-05-04 00:00:00,16.5,12.0,20.9,0.0,null,12.1,1018.0
2021-05-05 00:00:00,16.7,12.3,21.7,0.0,null,13.8,1016.5
…,…,…,…,…,…,…,…
2023-05-14 00:00:00,19.4,14.1,25.5,0.0,null,12.6,1018.2
2023-05-15 00:00:00,23.8,15.9,32.0,0.0,null,15.5,1015.5
2023-05-16 00:00:00,22.3,17.8,26.2,0.0,null,17.6,1013.0


In [89]:
print(weather.head(5))
print(weather.tail(5))

shape: (5, 8)
┌─────────────────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ time                ┆ tavg_w ┆ tmin_w ┆ tmax_w ┆ prcp_w ┆ wdir_w ┆ wspd_w ┆ pres_w │
│ ---                 ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ datetime[ns]        ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    │
╞═════════════════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 2021-05-01 00:00:00 ┆ 17.2   ┆ 11.9   ┆ 22.8   ┆ 0.0    ┆ null   ┆ 14.2   ┆ 1014.0 │
│ 2021-05-02 00:00:00 ┆ 16.3   ┆ 10.8   ┆ 23.2   ┆ 0.0    ┆ null   ┆ 15.7   ┆ 1015.9 │
│ 2021-05-03 00:00:00 ┆ 15.3   ┆ 12.7   ┆ 20.1   ┆ 6.5    ┆ null   ┆ 9.7    ┆ 1017.7 │
│ 2021-05-04 00:00:00 ┆ 16.5   ┆ 12.0   ┆ 20.9   ┆ 0.0    ┆ null   ┆ 12.1   ┆ 1018.0 │
│ 2021-05-05 00:00:00 ┆ 16.7   ┆ 12.3   ┆ 21.7   ┆ 0.0    ┆ null   ┆ 13.8   ┆ 1016.5 │
└─────────────────────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┘
shape: (5, 8)
┌──────────────

In [ ]:
# Para ver los nulos con Polars es algo más complejo, esta es la metodología
nulls_column = weather.select([
    pl.col(col).null_count().alias(col) for col in weather.columns
    ])

print(nulls_column)


shape: (1, 8)
┌──────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ time ┆ tavg_w ┆ tmin_w ┆ tmax_w ┆ prcp_w ┆ wdir_w ┆ wspd_w ┆ pres_w │
│ ---  ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ u32  ┆ u32    ┆ u32    ┆ u32    ┆ u32    ┆ u32    ┆ u32    ┆ u32    │
╞══════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 0    ┆ 0      ┆ 0      ┆ 0      ┆ 7      ┆ 748    ┆ 0      ┆ 0      │
└──────┴────────┴────────┴────────┴────────┴────────┴────────┴────────┘


In [ ]:
# Días en qué hay valores nulos
days_with_nulls = weather.filter(
    pl.any_horizontal([pl.col(c).is_null() for c in weather.columns])
    )
print(days_with_nulls)

# Se entiende que todos, ya que la dirección del viento es nula en todas las filas

shape: (748, 8)
┌─────────────────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ time                ┆ tavg_w ┆ tmin_w ┆ tmax_w ┆ prcp_w ┆ wdir_w ┆ wspd_w ┆ pres_w │
│ ---                 ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ datetime[ns]        ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    │
╞═════════════════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 2021-05-01 00:00:00 ┆ 17.2   ┆ 11.9   ┆ 22.8   ┆ 0.0    ┆ null   ┆ 14.2   ┆ 1014.0 │
│ 2021-05-02 00:00:00 ┆ 16.3   ┆ 10.8   ┆ 23.2   ┆ 0.0    ┆ null   ┆ 15.7   ┆ 1015.9 │
│ 2021-05-03 00:00:00 ┆ 15.3   ┆ 12.7   ┆ 20.1   ┆ 6.5    ┆ null   ┆ 9.7    ┆ 1017.7 │
│ 2021-05-04 00:00:00 ┆ 16.5   ┆ 12.0   ┆ 20.9   ┆ 0.0    ┆ null   ┆ 12.1   ┆ 1018.0 │
│ 2021-05-05 00:00:00 ┆ 16.7   ┆ 12.3   ┆ 21.7   ┆ 0.0    ┆ null   ┆ 13.8   ┆ 1016.5 │
│ …                   ┆ …      ┆ …      ┆ …      ┆ …      ┆ …      ┆ …      ┆ …      │
│ 2023-05-14 00:00:00 ┆ 19.

In [ ]:
# Fechas específicas dónde hay valores nulos en "prcp_w"
prcp_null = weather.filter(pl.col("prcp_w").is_null())  
print(prcp_null[["time", "prcp_w"]])


shape: (7, 2)
┌─────────────────────┬────────┐
│ time                ┆ prcp_w │
│ ---                 ┆ ---    │
│ datetime[ns]        ┆ f64    │
╞═════════════════════╪════════╡
│ 2021-10-26 00:00:00 ┆ null   │
│ 2021-10-30 00:00:00 ┆ null   │
│ 2022-01-27 00:00:00 ┆ null   │
│ 2022-02-28 00:00:00 ┆ null   │
│ 2022-03-05 00:00:00 ┆ null   │
│ 2022-04-26 00:00:00 ┆ null   │
│ 2022-04-27 00:00:00 ┆ null   │
└─────────────────────┴────────┘


In [ ]:
# Vemos todas las columnas de los días dónde las precipitaciones són nulas
print(prcp_null)

shape: (7, 8)
┌─────────────────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ time                ┆ tavg_w ┆ tmin_w ┆ tmax_w ┆ prcp_w ┆ wdir_w ┆ wspd_w ┆ pres_w │
│ ---                 ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ datetime[ns]        ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    ┆ f64    │
╞═════════════════════╪════════╪════════╪════════╪════════╪════════╪════════╪════════╡
│ 2021-10-26 00:00:00 ┆ 18.3   ┆ 15.6   ┆ 24.1   ┆ null   ┆ null   ┆ 9.6    ┆ 1020.4 │
│ 2021-10-30 00:00:00 ┆ 21.8   ┆ 17.6   ┆ 26.8   ┆ null   ┆ null   ┆ 6.8    ┆ 1014.6 │
│ 2022-01-27 00:00:00 ┆ 15.2   ┆ 14.0   ┆ 16.3   ┆ null   ┆ null   ┆ 29.2   ┆ 1024.9 │
│ 2022-02-28 00:00:00 ┆ 13.6   ┆ 10.9   ┆ 17.0   ┆ null   ┆ null   ┆ 9.6    ┆ 1026.8 │
│ 2022-03-05 00:00:00 ┆ 13.9   ┆ 7.9    ┆ 19.2   ┆ null   ┆ null   ┆ 12.4   ┆ 1016.2 │
│ 2022-04-26 00:00:00 ┆ 16.3   ┆ 13.5   ┆ 21.4   ┆ null   ┆ null   ┆ 8.7    ┆ 1013.8 │
│ 2022-04-27 00:00:00 ┆ 16.2 

In [ ]:
# Ruta para guardar archivos de datos ya procesados
PROCESSED_DATA_DIR = r"d:\PersonalProjects\Panadería Datathon\data\processed"

# Voy a convertir el archivo a csv también, ya que Polars tiene método directo
weather.write_csv(os.path.join(PROCESSED_DATA_DIR, "variables_meteorologicas.csv"))

# En Polars no hay método directo para convertir a Excel
weather_pandas = weather.to_pandas()

weather_pandas.to_excel(os.path.join(PROCESSED_DATA_DIR, "variables_meteorologicas.xlsx"))



## 3. Join -> Datos de ventas y del tiempo

In [104]:
ventas

familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia
str,i64,date,str,f64,f64,f64,i64
"""BOLLERIA""",3880,2021-05-01,null,2.591,1710.0,4430.609985,3
"""BOLLERIA""",3960,2021-05-01,null,2.318,2412.0,5591.015968,1
"""BOLLERIA""",5803,2021-05-01,null,2.727,1422.0,3877.793884,5
"""BOLLERIA""",6286,2021-05-01,null,3.136,990.0,3104.640038,4
"""BOLLERIA""",6425,2021-05-01,null,31.364,190.349997,5970.1325,2
…,…,…,…,…,…,…,…
"""PASTELERIA""",6523,2023-05-17,null,40.772999,72.0,2935.656006,3
"""PASTELERIA""",5403,2023-05-18,null,40.772999,46.800001,1908.174042,2
"""PASTELERIA""",5404,2023-05-18,null,43.5,108.0,4698.0,1


In [108]:
weather

time,tavg_w,tmin_w,tmax_w,prcp_w,wdir_w,wspd_w,pres_w
date,f64,f64,f64,f64,f64,f64,f64
2021-05-01,17.2,11.9,22.8,0.0,null,14.2,1014.0
2021-05-02,16.3,10.8,23.2,0.0,null,15.7,1015.9
2021-05-03,15.3,12.7,20.1,6.5,null,9.7,1017.7
2021-05-04,16.5,12.0,20.9,0.0,null,12.1,1018.0
2021-05-05,16.7,12.3,21.7,0.0,null,13.8,1016.5
…,…,…,…,…,…,…,…
2023-05-14,19.4,14.1,25.5,0.0,null,12.6,1018.2
2023-05-15,23.8,15.9,32.0,0.0,null,15.5,1015.5
2023-05-16,22.3,17.8,26.2,0.0,null,17.6,1013.0


In [ ]:
# Las columnas de join tienen que tener el mismo tipo, "fecha_venta" y "time"
ventas = ventas.with_columns(pl.col("fecha_venta").cast(pl.Date))

# El método .cast en Polars, convierte las columnas en el tipo que le indicamos
weather = weather.with_columns(pl.col("time").cast(pl.Date))

df_join = ventas.join(
    weather,
    left_on="fecha_venta",
    right_on="time",
    how="inner"
)

    

# Ordenar el df por "articulo" y la "fecha_venta"
df = df_join.sort(["articulo", "fecha_venta"])
df.head(5)

familia,articulo,fecha_venta,festivo,precio,cantidad,importe,orden_articulo_familia,tavg_w,tmin_w,tmax_w,prcp_w,wdir_w,wspd_w,pres_w
str,i64,date,str,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
"""PANADERIA""",417,2021-05-01,null,4.038,432.0,1744.416023,5,17.2,11.9,22.8,0.0,null,14.2,1014.0
"""PANADERIA""",417,2021-05-02,"""Día de la Madre""",4.038,486.0,1962.467972,5,16.3,10.8,23.2,0.0,null,15.7,1015.9
"""PANADERIA""",417,2021-05-03,"""Día de la Cruz""",4.038,540.0,2180.519989,5,15.3,12.7,20.1,6.5,null,9.7,1017.7
"""PANADERIA""",417,2021-05-04,null,4.038,504.0,2035.152008,5,16.5,12.0,20.9,0.0,null,12.1,1018.0
"""PANADERIA""",417,2021-05-05,null,4.038,504.0,2035.15202,5,16.7,12.3,21.7,0.0,null,13.8,1016.5


## 4. EDA

******

#### 4.1. Inspección inicial

Este apartado tiene como cometido comprender la estructura del conjunto de datos y ver posibles problemas de calidad que podrían afectar al análisis.

**Grupo de preguntas 1** (x minutos):

1. Describe que significa cada fila de nuestro conjunto de datos.

2. ¿Cuántos valores únicos hay en cada una de las variables? ¿Qué insight podrías observar al comparar los valores únicos de la variable "articulo" con los valores únicos de la variable "precio"?

3. ¿Cuántos valores nulos hay en cada una de las variables?

4. ¿Hay duplicados?

In [ ]:
# 2
df.describe(include='all')

In [ ]:
df['familia'] = df['familia'].astype(str)
df['festivo'] = df['festivo'].astype(str)

In [ ]:
# 2
# La diferencia entre el número único de artículos y de precios muestra que los precios cambian a lo largo del tiempo
summary = (
    df.dtypes.to_frame("Tipo")
    .assign(Cardinalidad = df.nunique())
    .assign(Granularidad = df.nunique() / len(df) * 100)
    .assign(Nulos_Pct = df.isnull().sum() / len(df) * 100)
    # .assign(Max = df.dropna().apply(max))
    # .assign(Min = df.dropna().apply(min))
    # .sort_values(["Tipo"])
)

In [ ]:
# Calcular máximos y mínimos por separado y unirlos
max_vals = df.max(numeric_only=False).rename("Max")
min_vals = df.min(numeric_only=False).rename("Min")

# Unir todo
summary = summary.join([max_vals, min_vals])



In [ ]:
summary.sort_values("Tipo", key=lambda col: col.astype(str))

In [ ]:
# 3
# Valores nulos

msno.matrix(df)

In [ ]:
# 4
# Valores duplicados

df[df.duplicated(['fecha_venta', 'articulo'], keep=False)]
#df.drop_duplicates(['fecha_venta', 'articulo'])

**Grupo de preguntas 2** (x minutos):

5. ¿Cuál es el rango de fechas de nuestro conjunto de datos? Si se divide por producto, ¿hay fechas faltantes? Crea un gráfico de evolución temporal para la variable "cantidad" que muestre el producto "6549".

6. Separando por producto, ¿hay outliers en la variable "cantidad"?

In [ ]:
# 5
# Rango de fechas del conjunto de datos

min_date = df['fecha_venta'].min()
max_date = df['fecha_venta'].max()

print(f"El conjunto de datos contiene valores desde {min_date} hasta {max_date}. ",
      f"Lo que supone {max_date - min_date}")

In [ ]:
# 5
# Fechas faltantes

unique_articles = df.sort_values("familia")["articulo"].unique().tolist()
complete_range = pd.date_range(start=min_date, end=max_date, freq='D')

for article in unique_articles:

    subset = df.query("articulo == @article")
    family = subset["familia"].unique().tolist()[0]

    missing_dates = complete_range.difference(subset['fecha_venta'])

    print(family, "-> Articulo", article, "-> Fechas faltantes", len(missing_dates))

In [ ]:
# 5
# Gráfico de la cantidad del producto "6549"

articulo = 3960
filtered_df = (
    df.query("articulo == @articulo")
      .set_index("fecha_venta")
      .reindex(complete_range)
)

filtered_df["articulo"] = filtered_df["articulo"].fillna(method='ffill')

fig = px.line(filtered_df, x=filtered_df.index, y="cantidad", color="articulo")
fig.show()

In [ ]:
articulo = 5403
filtered_df = (
    df.query("articulo == @articulo")
      .set_index("fecha_venta")
      .reindex(complete_range)
)

filtered_df["articulo"] = filtered_df["articulo"].fillna(method='ffill')

fig = px.line(filtered_df, x=filtered_df.index, y="cantidad", color="articulo")
fig.show()

In [ ]:
# 6
# Outliers en cantidad desglosando por producto

unique_articles = df.sort_values("familia")["articulo"].unique().tolist()

for article in unique_articles:

    subset = df.query("articulo == @article")
    family = subset["familia"].unique().tolist()[0]

    Q1 = subset['cantidad'].quantile(0.25)
    Q3 = subset['cantidad'].quantile(0.75)
    IQR = Q3 - Q1

    outliers = subset[(subset['cantidad'] < (Q1 - 1.5 * IQR)) | (subset['cantidad'] > (Q3 + 1.5 * IQR))]

    print(family, "-> Articulo", article, "-> Outliers", len(outliers["cantidad"]))

In [ ]:
  outliers.head()

*****

#### 4.2. Análisis de la variable a predecir

Este apartado tiene como cometido comprender la evolución de la variable "cantidad" en el tiempo y como se relaciona esta consigo misma.

**Grupo de preguntas 3** (x minutos):

7. Crea un gráfico de la evolución temporal general de la variable "cantidad". Nota: Se debe de agrupar.

8. Crea un gráfico de la evolución temporal por familia de la variable "cantidad". Nota: Se debe de agrupar.

9. Crea un gráfico de la evolución temporal por artículo de la variable "cantidad". Nota: Se debe de agrupar.

10. Se que hay mucho ruido, pero ¿a simple vista crees que hay tendencia y/o estacionalidad en las series temporales anteriores?

In [ ]:
# 7
# Evolución temporal general de la variable "cantidad"

group = df.groupby(pd.Grouper(key="fecha_venta", freq="1D"))["cantidad"].sum().reset_index()
fig = px.line(group, x="fecha_venta", y="cantidad")
fig.show()

In [ ]:
# 8
# Evolución temporal por familia de la variable "cantidad"

group = df.groupby(["familia", pd.Grouper(key="fecha_venta", freq="1D")])["cantidad"].sum().reset_index()
fig = px.line(group, x="fecha_venta", y="cantidad", color="familia")
fig.show()

In [ ]:
# 8
# Evolución temporal por familia de la variable "cantidad"

scaler = MinMaxScaler()

def normalize(column):
    return scaler.fit_transform(column.values.reshape(-1, 1)).flatten()

group = df.groupby(["familia", pd.Grouper(key="fecha_venta", freq="1D")])["cantidad"].sum().reset_index()
group['cantidad'] = group.groupby('familia')['cantidad'].transform(normalize)
fig = px.line(group, x="fecha_venta", y="cantidad", color="familia")
fig.show()

In [ ]:
# 9
# Evolución temporal por artículo de la variable "cantidad"

group = df.groupby(["articulo", pd.Grouper(key="fecha_venta", freq="1D")])["cantidad"].sum().reset_index()
fig = px.line(group, x="fecha_venta", y="cantidad", color="articulo")
fig.show()

**Grupo de preguntas 4** (x minutos):

11. Aplica alguna técnica estadística para observar si hay estacionalidad en la evolucion temporal general de la variable "cantidad". Pista: Tomar la primera diferencia y, después, hacer un analisis de autocorrelación.

12. Sin aplicar la primera diferencia y creando nuevas columnas de fecha a partir de la variable "fecha_venta" (semana del año, mes del año, día de la semana, día del mes, día del año) comprueba realizando diferentes agrupaciones y gráficos si nuestro análisis de autocorrelación de nuestra variable cantidad nos mostraba lo correcto. Si encuentras algún gráfico que te llame la atención, baja el nivel del análisis (Ej: 1. Cantidad general -> 2. Cantidad por familia -> 3. Cantidad por artículo).

In [ ]:
# 11
# Check si el proceso es estacionario

def check_stationarity(series):
    result = adfuller(series.values)

    print('ADF Statistic: ', result[0])
    print('p-value: ', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32m Stationary \u001b[0m")
    else:
        print("\x1b[31m Non-stationary \x1b[0m")

group = df.groupby([pd.Grouper(key="fecha_venta", freq="1D")])["cantidad"].sum().reset_index()
check_stationarity(group["cantidad"])
fig = px.line(group, x="fecha_venta", y="cantidad")
fig.show()

In [ ]:
# 11
# Tomando la primera diferencia

# Podemos tomar la primera diferencia para observar si nuestra serie se convierte en estacionaria.
# Esto elimina tendencias y estabiliza la varianza de la serie, permitiendo que sea modelada.

group["cantidad_diff"] = group["cantidad"].diff().fillna(0)
check_stationarity(group["cantidad_diff"])

In [ ]:
# 11
# La nueva serie temporal tiene esta pinta

fig = px.line(group, x="fecha_venta", y="cantidad_diff")
fig.show()

In [ ]:
# 11
# Analisis de autocorrelación

# En terminos generales, este gráfico nos indica que un valor de cantidad esta
# directamente relacionado con su valor de cantidad anterior y su valor de cantidad de la semana pasada.
# Parece que hay estacionalidad semanal.

plot_acf(group["cantidad_diff"],lags=56)
plt.tight_layout()
plt.show()

In [ ]:
# 12

# Semana y mes del año
df["weekofyear"] = df["fecha_venta"].dt.isocalendar().week
df["monthofyear"] = df["fecha_venta"].dt.month

# Día de la semana, del mes y del año
df["dayofweek"] = df["fecha_venta"].dt.b #Lunes 0 y Domingo 6
df["dayofmonth"] = df["fecha_venta"].dt.day
df["dayofyear"] = df["fecha_venta"].dt.dayofyear

def graph_by_freq(freq, breakdown="familia", norm=False):
    group = df.groupby([breakdown, freq], as_index=False)["cantidad"].sum()

    if norm == True:
        group['cantidad'] = group.groupby(breakdown)['cantidad'].transform(normalize)

    fig = px.line(group, x=freq, y="cantidad", color=breakdown, markers=True)
    fig.show()

# A nivel familia
graph_by_freq("dayofweek")

# Como nos encontramos en nuestro análisis de autocorrelación,
# los valores de cantidad podrían tener una relación con su valor de la semana pasada
# debido a que hay un patrón de compra semanal que se repite en el tiempo.

In [ ]:
# 12

# A nivel articulo
graph_by_freq("dayofweek", "articulo")

In [ ]:
# 12
# Gráficos caja

fig = px.box(df, x="dayofweek", y="cantidad", color='familia')
fig.show()

In [ ]:
graph_by_freq("monthofyear")

*****

#### 4.3. Análisis de la variable a predecir frente al resto de variables

**Grupo de preguntas 5** (x minutos):

13. ¿El comportamiento de compra (la variable "cantidad") cuando es festivo es superior a cuando no lo es?

14. ¿El comportamiento de compra (la variable "cantidad") cuando llueve es superior a cuando no llueve?

15. Divide la variable "tavg_w" en quintiles y muestra con un gráfico de barras sí la variable "cantidad" es superior en alguno de sus quintiles.

In [ ]:
# 13

# 1 si es festivo, 0 si no lo es
df['es_festivo'] = df['festivo'].apply(lambda x: 0 if x is None else 1)

fig = px.box(df, x="es_festivo", y="cantidad", color='familia')
fig.show()

# El comportamiento de compra cuando es festivo parece ligeramente superior

In [ ]:
# 14

# 1 si llueve, 0 si no llueve
df["lluvia"] = np.where(df["prcp_w"] > 0, 1, 0)

fig = px.box(df, x="lluvia", y="cantidad", color='familia')
fig.show()

# Parece que la lluvia no es un impedimiento para comprar

In [ ]:
# 15

labels_tavg = ['Temperatura muy baja',
               'Temperatura baja',
               'Temperatura normal',
               'Temperatura alta',
               'Temperatura muy alta']

df['quintiles_tavg_w'] = pd.qcut(df['tavg_w'], q=5, labels=labels_tavg)

group = df.groupby(["familia", "quintiles_tavg_w"])["cantidad"].sum().reset_index()
fig = px.bar(group, x='quintiles_tavg_w', y='cantidad', color="familia",barmode='group')
fig.show()

**Grupo de preguntas 6** (x minutos):

16. ¿Un incremento en el precio reduce la propensión a consumir de un artículo?

In [ ]:
# 16

def function_variations(x):
    list_unique = x.unique()
    len_list = len(list_unique)

    if len_list > 1:
        return 1
    else:
        return 0

group = df.groupby(["articulo",
                    pd.Grouper(key="fecha_venta", freq="1M"),
                    "familia"]).agg({"cantidad": "sum",
                                     "precio": function_variations}).reset_index()

group["precio"] = group.groupby("articulo")["precio"].transform("cumsum")
group["precio"] = group["precio"].astype(object)

fig = px.box(group, x="precio", y="cantidad", color='articulo')
fig.show()

# No es concluyente debido a que puede haber una tendencia negativa/positiva en el consumo general del articulo
# a lo largo del tiempo, pero es interesante observar como muchas veces si que tiene un impacto negativo
# (productos: 417, 1043, 1084, 3960, 5403).
# Podría ser también por la canibalización de nuevos productos a otros anteriores.

*****